# 데이터 정제

1. pandas, matplotlib, numpy를 가져와서 ufos 스프레드시트를 가져옵니다. 샘플 데이터 집합을 살펴볼 수 있습니다:

In [1]:
import pandas as pd
import numpy as np
ufos = pd.read_csv('https://raw.githubusercontent.com/JOJeongHaeng/jeonghaeng.github.io/main/ufos.csv')
ufos.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


2. ufos 데이터를 새로운 제목을 가진 작은 데이터 프레임으로 변환합니다. 국가 필드에서 고유 값을 확인합니다.

In [2]:
ufos = pd.DataFrame({'Seconds': ufos['duration (seconds)'], 'Country': ufos['country'],'Latitude': ufos['latitude'],'Longitude': ufos['longitude']})

ufos.Country.unique()

array(['us', nan, 'gb', 'ca', 'au', 'de'], dtype=object)

이제 null 값을 삭제하고 1~60초 사이의 목격 횟수만 가져와서 처리해야 하는 데이터의 양을 줄일 수 있습니다:

In [3]:
ufos.dropna(inplace=True)

ufos = ufos[(ufos['Seconds'] >= 1) & (ufos['Seconds'] <= 60)]

ufos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25863 entries, 2 to 80330
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Seconds    25863 non-null  float64
 1   Country    25863 non-null  object 
 2   Latitude   25863 non-null  float64
 3   Longitude  25863 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1010.3+ KB


Scikit-learn의 LabelEncoder 라이브러리를 가져와서 국가별 텍스트 값을 숫자로 변환합니다:

라벨인코더는 데이터를 알파벳순으로 인코딩합니다.

In [4]:
from sklearn.preprocessing import LabelEncoder

ufos['Country'] = LabelEncoder().fit_transform(ufos['Country'])

ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,3,53.200000,-2.916667
3,20.0,4,28.978333,-96.645833
14,30.0,4,35.823889,-80.253611
23,60.0,4,45.582778,-122.352222
24,3.0,3,51.783333,-0.783333


#모델 구축

이제 데이터를 학습 그룹과 테스트 그룹으로 나누어 모델 학습을 준비할 수 있습니다.

1. 훈련하려는 세 가지 기능을 X 벡터로 선택하고, Y 벡터는 국가가 됩니다. 초, 위도 및 경도를 입력하고 반환할 국가 ID를 가져올 수 있기를 원합니다.

In [5]:
from sklearn.model_selection import train_test_split

Selected_features = ['Seconds','Latitude','Longitude']

X = ufos[Selected_features]
y = ufos['Country']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

2. 로지스틱 회귀를 사용하여 모델을 훈련합니다:

In [6]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.83      0.23      0.36       250
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00       131
           4       0.96      1.00      0.98      4743

    accuracy                           0.96      5173
   macro avg       0.96      0.85      0.87      5173
weighted avg       0.96      0.96      0.95      5173

Predicted labels:  [4 4 4 ... 3 4 4]
Accuracy:  0.9605644693601392


C:\Users\goddl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


국가와 위도/경도가 상관관계가 있기 때문에 정확도는 당연히 나쁘지 않습니다(약 95%).

위도와 경도를 통해 국가를 유추할 수 있어야 하기 때문에 만든 모델이 그다지 혁신적이지는 않지만, 정리하고 내보낸 원시 데이터로 학습한 다음 웹 앱에서 이 모델을 사용해 보는 것은 좋은 연습이 됩니다.

#모델 '피클'

이제 모델을 피클할 시간입니다! 몇 줄의 코드로 이 작업을 수행할 수 있습니다. 피클이 완료되면 피클된 모델을 로드하고 초, 위도 및 경도 값이 포함된 샘플 데이터 배열에 대해 테스트합니다,

In [7]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(model, open(model_filename,'wb'))

model = pickle.load(open('ufo-model.pkl','rb'))
print(model.predict([[50,44,-12]]))

[1]


C:\Users\goddl\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
